# Importing Libraries

In [1]:
import os
from dotenv import load_dotenv
import random
import numpy as np
from dataclasses import dataclass

# PyTorch
import torch

# Huggingface
import huggingface_hub
from transformers import TextStreamer

# Unsloth
from unsloth import FastLanguageModel, is_bf16_supported

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


# Configuration

In [2]:
@dataclass
class CONFIG:
    # Model
    model_id: str = "PathFinderKR/KHU-Llama-3.2-1B-Instruct-SFT"
    model_type: str = "instruct"
    
    # Inference
    max_new_tokens: int = 2048
    do_sample: bool = True
    temperature: float = 0.7
    top_p: float = 0.9
    repetition_penalty: float = 1.1
    dtype: torch.dtype = torch.bfloat16 if is_bf16_supported() else torch.float16
    load_in_4bit: bool = True
    
    # Device
    device: torch.device = None
    
    # Seed
    seed: int = 42

## Reproducibility

In [3]:
def set_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    print(f"Seed: {seed}")
    
set_seed(CONFIG.seed)

Seed: 42


## Device

In [4]:
def configure_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        num_gpu = torch.cuda.device_count()
        print("> Running on GPU", end=' | ')
        print("Num of GPUs: ", num_gpu)
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
        print("> Running on MPS")
    else:
        device = torch.device("cpu")
        print("> Running on CPU")
    return device

CONFIG.device = configure_device()

> Running on GPU | Num of GPUs:  1


## HuggingFace

In [5]:
load_dotenv()
huggingface_hub.login(
    token=os.getenv("HUGGINGFACE_TOKEN"),
    add_to_git_credential=True
)

# Model

In [6]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=CONFIG.model_id,
    max_seq_length=CONFIG.max_new_tokens,
    dtype=CONFIG.dtype,
    load_in_4bit=CONFIG.load_in_4bit
)

==((====))==  Unsloth 2024.11.8: Fast Llama patching. Transformers = 4.46.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4080 SUPER. Max memory: 15.992 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

<string>:204: SyntaxWarning: invalid escape sequence '\ '
<string>:205: SyntaxWarning: invalid escape sequence '\_'
<string>:206: SyntaxWarning: invalid escape sequence '\ '
<string>:204: SyntaxWarning: invalid escape sequence '\ '
<string>:205: SyntaxWarning: invalid escape sequence '\_'
<string>:206: SyntaxWarning: invalid escape sequence '\ '


In [7]:
FastLanguageModel.for_inference(model)
text_streamer = TextStreamer(tokenizer)

# Inference

In [8]:
llama_3_instruct_prompt = """<|start_header_id|>system<|end_header_id|>

{}<|eot_id|><|start_header_id|>user<|end_header_id|>

{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{}"""

# Generate base model
def generate_text(prompt):
    FastLanguageModel.for_inference(model)
    inputs = tokenizer(
    [
        prompt
    ], return_tensors = "pt").to(CONFIG.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=CONFIG.max_new_tokens,
        do_sample=CONFIG.do_sample,
        temperature=CONFIG.temperature,
        top_p=CONFIG.top_p,
        repetition_penalty=CONFIG.repetition_penalty,
        use_cache=True,
        streamer=TextStreamer(tokenizer)
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=False)

# Generate instruction model
def generate_response(system, user):
    FastLanguageModel.for_inference(model)
    inputs = tokenizer(
    [
        llama_3_instruct_prompt.format(
            system,
            user,
            ""
        )
    ], return_tensors = "pt").to(CONFIG.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=CONFIG.max_new_tokens,
        do_sample=CONFIG.do_sample,
        temperature=CONFIG.temperature,
        top_p=CONFIG.top_p,
        repetition_penalty=CONFIG.repetition_penalty,
        use_cache=True,
        streamer=TextStreamer(tokenizer)
    )
    return tokenizer.batch_decode(outputs, skip_special_tokens=False)

In [11]:
if CONFIG.model_type == "base":
    sample_text = "Capital of France:"
    sample_response = generate_text(sample_text)
    print(sample_response)
    print(tokenizer.tokenize(sample_response[0]))
elif CONFIG.model_type == "instruct":
    sample_system = "You are a helpful assistant."
    sample_user = "How are final exams structured in June?"
    sample_response = generate_response(sample_system, sample_user)
    print(sample_response)
    print(tokenizer.tokenize(sample_response[0]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

How are final exams structured in June?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

In the month of June, students typically take several final exams as part of their academic progression. Here’s how they are structured and what to expect:\n\n1. **Final Exams Structure**: \n   - Students usually take multiple courses, each with its own set of subjects or credits.\n   - The course structure may vary by institution; some might have a single subject while others offer multiple subjects.\n   - For example, a student taking 'Physics 101' might also encounter 'Physics 102', which could be worth additional credit towards their degree.\n\n2. **Timing of Final Exams**:
   - These exams often occur in the last semester of the year, specifically between December 15th and January 20th.\n   - This timing allows for proper grading and feedback befor